Make sure to `pip install hud-python[agents]` before running this notebook

### Step 1: Create a Task

A Task combines:
- **Prompt**: What we want an agent to accomplish
- **MCP Config**: How to spawn the environment
- **Setup Tool**: How to prepare the environment
- **Evaluate Tool**: How to check if the task succeeded

In [ ]:
from hud.datasets import Task
from hud.types import MCPToolCall

# Create a task that uses our test_test environment
# See tasks.json for how to build a loadable task dataset
task = Task(
    prompt="Increment the counter to reach 10",
    mcp_config={
        "test_test": {"url": "http://localhost:8765/mcp"},
    },
    setup_tool=MCPToolCall(name="setup", arguments={}),
    evaluate_tool=MCPToolCall(name="evaluate", arguments={"target": 10}),
)

### Step 2: Initialize MCP Client

Run `hud dev --build` before this cell to intialize the server at `http://localhost:8765/mcp`

In [ ]:
from hud.clients import MCPClient

# Create the client
client = MCPClient(mcp_config=task.mcp_config, auto_trace=False)

# Initialize it (this connects to our dev server)
await client.initialize()

### Step 3: Run Setup

Call the setup tool to prepare the environment according to the task.

In [ ]:
# Run the setup from our task
setup_result = await client.call_tool(task.setup_tool)  # type: ignore
print(f"Setup result: {setup_result}")

### Step 4: Perform Actions

Now we'll manually perform actions to complete the task. In a real scenario, an AI agent would figure out what actions to take.

In [ ]:
# Increment the counter 10 times
for i in range(10):
    result = await client.call_tool(name="act", arguments={})
    print(f"Step {i + 1}: {result.content}")

## Step 5: Evaluate Success

Check if we completed the task according to the evaluation criteria.

In [ ]:
# Run the evaluation from our task
eval_result = await client.call_tool(task.evaluate_tool)  # type: ignore

# The result is a list with one TextContent item containing JSON
print(eval_result)

### Step 6: Cleanup

Always shut down the client when done to stop the Docker container. Either stop hud dev in the terminal, or run this command:

In [ ]:
await client.shutdown()

### Bonus: Running with an AI Agent

Instead of manually calling tools, you can have an AI agent solve the task automatically.

In [ ]:
# Uncomment to run with Claude (requires ANTHROPIC_API_KEY)
from hud.agents import ClaudeAgent

# Create an agent
agent = ClaudeAgent(
    model="claude-sonnet-4-20250514",
    allowed_tools=["act"],  # Only allow the act tool
)

# Run the task
result = await agent.run(task)
print(f"Final reward: {result.reward}")

### Next Steps

1. **Create your own evaluators**: Add new evaluation functions to `server.py`
2. **Build complex environments**: Replace the simple counter with your actual application
3. **Test with agents**: Use different AI models to solve your tasks

For more examples, check out:
- `environments/text_2048/` - A complete 2048 game environment
- `environments/browser/` - A full browser automation environment with GUI